In [25]:
from scipy.sparse import csr_matrix, lil_matrix
from scipy.io import mmread
import numpy as np
import pandas as pd
import utils
import scores
from math import pi
from random import randint

In [4]:
data, adjacency, edges = utils.loadData("small")

In [5]:
solution = data["assignment"]

In [4]:
solution_perimeter = scores.perimeter(data, adjacency, edges, solution)

In [8]:
area = data["area"].groupby(solution).sum()

In [10]:
4*pi*area/(solution_perimeter**2)

assignment
1    0.268645
2    0.248994
3    0.231095
dtype: float64

In [9]:
area/solution_perimeter

assignment
1    519.024053
2    713.145716
3    809.003866
dtype: float64

In [18]:
np.unique(solution)

array([1, 2, 3])

In [20]:
pd.Series(np.zeros(len(np.unique(solution))), index=np.unique(solution))

1    0.0
2    0.0
3    0.0
dtype: float64

In [13]:
area

assignment
1    1.260104e+07
2    2.566717e+07
3    3.558940e+07
Name: area, dtype: float64

# Initializer

In [49]:
data, adjacency, edges = utils.loadData("SD")

In [67]:
# How many total units to assign
n = data.shape[0]
# How many districts to build
k = 5
# Solution array (initially nothing is assigned)
sol = np.zeros(n, dtype="int32")
# Select k seed zones as starting elements
seeds = data[data["outer_edge"] > 0].sample(k).index.values

# Set assignment for each of the seeds
for i, v in enumerate(seeds):
    sol[v] = i+1



In [68]:
pool = seeds.tolist()

In [69]:
# While there are still elements in pool
while pool:
    # Select an index available in pool
    i = randint(0, len(pool)-1)
    # Pop that item out
    u = pool.pop(i)
    # Get list of zones adjacent to u
    for v in adjacency.rows[u]:
        if sol[v] == 0 and v not in pool:
            pool.append(v)
            sol[v] = sol[u]
    

In [70]:
print(", ".join(map(str,sol)))

5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 1, 1, 1, 5, 5, 5, 5, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5

In [33]:
adjacency.rows[u]

[4, 5, 7]